# AILib Tutorial 3: Prompt Templates

Prompt templates are a powerful feature in AILib that allow you to create reusable, dynamic prompts. In this tutorial, you'll learn:

- Creating basic prompt templates
- Using variables and placeholders
- Default values and validation
- Advanced formatting options
- Template composition
- Best practices for prompt engineering

## Setup

Let's start by importing what we need:

In [ ]:
from ailib import OpenAIClient
from ailib.prompts import PromptTemplate
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Create a client
client = OpenAIClient()
print("Ready to work with prompt templates!")

## Basic Prompt Templates

A prompt template is a string with placeholders that can be filled in later:

In [ ]:
# Create a simple template
template = PromptTemplate(
    "Translate the following {language} text to English: {text}"
)

# Format the template with values
prompt = template.format(
    language="Spanish",
    text="Hola, ¿cómo estás?"
)

print(f"Generated prompt: {prompt}")
print(f"\nLLM Response: {client.complete(prompt)}")

In [ ]:
# Reuse the same template with different values
languages = [
    ("French", "Bonjour, comment allez-vous?"),
    ("German", "Guten Tag, wie geht es Ihnen?"),
    ("Italian", "Ciao, come stai?")
]

for lang, text in languages:
    prompt = template.format(language=lang, text=text)
    response = client.complete(prompt)
    print(f"{lang}: {text}")
    print(f"English: {response}\n")

## Default Values

Templates can have default values for optional parameters:

In [ ]:
# Template with defaults
email_template = PromptTemplate(
    """Write a {tone} email to {recipient} about {topic}.
    
    Additional requirements: {requirements}""",
    # Default values
    tone="professional",
    requirements="Keep it concise and clear."
)

# Use with all defaults
prompt1 = email_template.format(
    recipient="the team",
    topic="the upcoming project deadline"
)
print("With defaults:")
print(prompt1)
print("\n" + "="*50 + "\n")

# Override defaults
prompt2 = email_template.format(
    recipient="the CEO",
    topic="quarterly results",
    tone="formal",
    requirements="Include specific metrics and highlight key achievements."
)
print("With overrides:")
print(prompt2)

## Template Validation

Templates validate that all required variables are provided:

In [ ]:
# Template with required and optional fields
story_template = PromptTemplate(
    """Write a {genre} story about {main_character}.
    Setting: {setting}
    Mood: {mood}""",
    mood="mysterious"  # Default value makes this optional
)

# This works - all required fields provided
try:
    prompt = story_template.format(
        genre="sci-fi",
        main_character="a time-traveling scientist",
        setting="Mars colony in 2150"
    )
    print("✓ Valid prompt created:")
    print(prompt)
except ValueError as e:
    print(f"✗ Error: {e}")

print("\n" + "="*50 + "\n")

# This fails - missing required field
try:
    prompt = story_template.format(
        genre="fantasy"
        # Missing main_character and setting!
    )
except ValueError as e:
    print(f"✗ Error caught: {e}")

## Complex Templates

Create sophisticated prompts with multiple sections:

In [ ]:
# Code review template
code_review_template = PromptTemplate(
    """You are an expert {language} developer performing a code review.
    
    Review the following code for:
    - Code quality and best practices
    - Potential bugs or issues
    - Performance considerations
    - Security vulnerabilities
    
    Code to review:
    ```{language}
    {code}
    ```
    
    Context: {context}
    Focus areas: {focus}
    
    Please provide specific, actionable feedback.""",
    language="Python",
    focus="General review"
)

# Example usage
code_to_review = """
def calculate_average(numbers):
    total = 0
    for n in numbers:
        total += n
    return total / len(numbers)
"""

prompt = code_review_template.format(
    code=code_to_review,
    context="This function is used in a financial application",
    focus="Error handling and edge cases"
)

response = client.complete(prompt)
print(response)

## Template Composition

Combine templates for more complex scenarios:

In [ ]:
# Base templates
persona_template = PromptTemplate(
    "You are a {role} with expertise in {expertise}. {additional_traits}"
)

task_template = PromptTemplate(
    "Your task is to {action} for {audience}. {constraints}"
)

format_template = PromptTemplate(
    "Format your response as {format}. {format_details}"
)

# Compose them together
def create_expert_prompt(role, expertise, action, audience, format_type):
    persona = persona_template.format(
        role=role,
        expertise=expertise,
        additional_traits="You communicate clearly and provide practical examples."
    )
    
    task = task_template.format(
        action=action,
        audience=audience,
        constraints="Keep it under 200 words."
    )
    
    format_spec = format_template.format(
        format=format_type,
        format_details="Use bullet points for key concepts."
    )
    
    return f"{persona}\n\n{task}\n\n{format_spec}\n\nTopic: "

# Create specialized prompts
teacher_prompt = create_expert_prompt(
    role="high school teacher",
    expertise="physics",
    action="explain quantum mechanics",
    audience="16-year-old students",
    format_type="an engaging lesson plan"
)

print("Composed prompt:")
print(teacher_prompt)
print("\n" + "="*50 + "\n")

response = client.complete(teacher_prompt + "the double-slit experiment")
print("Response:")
print(response)

## Dynamic Templates

Create templates that adapt based on conditions:

In [ ]:
# Function to create dynamic templates
def create_analysis_template(analysis_type, include_examples=True):
    base = "Analyze the following {content_type}: {content}\n\n"
    
    if analysis_type == "sentiment":
        base += "Determine the sentiment (positive, negative, neutral) and confidence level."
    elif analysis_type == "summary":
        base += "Provide a concise summary in {summary_length} sentences."
    elif analysis_type == "key_points":
        base += "Extract the {num_points} most important points."
    
    if include_examples:
        base += "\n\nProvide specific examples from the text to support your analysis."
    
    return PromptTemplate(base)

# Create different analysis templates
sentiment_template = create_analysis_template("sentiment")
summary_template = create_analysis_template("summary", include_examples=False)

# Example text
review = """The new smartphone has an amazing camera and battery life. 
However, the price is quite high and the design feels outdated. 
Overall, it's a decent phone but not worth the premium price tag."""

# Sentiment analysis
prompt1 = sentiment_template.format(
    content_type="product review",
    content=review
)
print("Sentiment Analysis:")
print(client.complete(prompt1))

print("\n" + "="*50 + "\n")

# Summary
prompt2 = summary_template.format(
    content_type="product review",
    content=review,
    summary_length=2
)
print("Summary:")
print(client.complete(prompt2))

## Template Libraries

Organize templates for reuse across your application:

In [ ]:
# Create a template library
class PromptLibrary:
    """Collection of reusable prompt templates."""
    
    # Writing templates
    blog_post = PromptTemplate(
        """Write a {word_count}-word blog post about {topic}.
        Target audience: {audience}
        Tone: {tone}
        Include: {requirements}""",
        word_count=500,
        tone="informative and engaging",
        requirements="introduction, main points, and conclusion"
    )
    
    social_media = PromptTemplate(
        """Create a {platform} post about {topic}.
        Tone: {tone}
        Include: {hashtags} relevant hashtags
        Character limit: {char_limit}""",
        hashtags=3,
        tone="casual and engaging"
    )
    
    # Analysis templates
    data_analysis = PromptTemplate(
        """Analyze this {data_type} data: {data}
        Focus on: {focus_areas}
        Provide: {deliverables}""",
        deliverables="key insights, trends, and recommendations"
    )
    
    # Coding templates
    code_generation = PromptTemplate(
        """Write a {language} function that {description}.
        Requirements:
        - {requirements}
        Include: {include}""",
        include="docstring, type hints, and error handling"
    )

# Use the library
lib = PromptLibrary()

# Generate a social media post
prompt = lib.social_media.format(
    platform="Twitter",
    topic="the benefits of prompt engineering",
    char_limit=280
)

response = client.complete(prompt)
print("Twitter Post:")
print(response)
print(f"\nCharacter count: {len(response)}")

## Advanced Formatting

Use advanced formatting techniques for better prompts:

In [ ]:
# Template with structured output
structured_template = PromptTemplate(
    """Analyze the business idea: {idea}
    
    Provide your analysis in the following JSON format:
    {{
        "viability_score": (1-10),
        "strengths": [list of strengths],
        "weaknesses": [list of weaknesses],
        "market_size": "estimate",
        "competition_level": "low/medium/high",
        "recommended_next_steps": [list of actions]
    }}
    
    Be specific and realistic in your assessment."""
)

prompt = structured_template.format(
    idea="An app that uses AI to help people plan personalized workout routines"
)

response = client.complete(prompt)
print(response)

In [ ]:
# Template with examples (few-shot prompting)
few_shot_template = PromptTemplate(
    """Convert natural language to SQL queries.
    
    Examples:
    Input: Show all users who joined last month
    Output: SELECT * FROM users WHERE created_at >= DATE_SUB(NOW(), INTERVAL 1 MONTH)
    
    Input: Count orders by status
    Output: SELECT status, COUNT(*) as count FROM orders GROUP BY status
    
    Input: Find the top 5 customers by total spending
    Output: SELECT customer_id, SUM(total) as total_spent FROM orders GROUP BY customer_id ORDER BY total_spent DESC LIMIT 5
    
    Now convert this:
    Input: {query}
    Output:"""
)

queries = [
    "Show all products under $50",
    "Get the average order value by month",
    "Find customers who haven't ordered in 90 days"
]

for query in queries:
    prompt = few_shot_template.format(query=query)
    sql = client.complete(prompt)
    print(f"Natural language: {query}")
    print(f"SQL: {sql}")
    print()

## Best Practices

Here are some tips for effective prompt templates:

In [ ]:
# 1. Use clear variable names
good_template = PromptTemplate(
    "Summarize this {document_type} about {topic} for {target_audience}"
)

# Not this:
# bad_template = PromptTemplate("Summarize {x} about {y} for {z}")

# 2. Include context and constraints
detailed_template = PromptTemplate(
    """Role: You are a {role}
    Task: {task}
    Constraints: {constraints}
    Format: {output_format}
    
    Input: {input_data}""",
    constraints="Be concise and factual",
    output_format="Paragraph form"
)

# 3. Use defaults wisely
flexible_template = PromptTemplate(
    """Generate {content_type} about {topic}.
    Length: {length}
    Style: {style}
    Additional notes: {notes}""",
    length="medium",
    style="professional",
    notes="None"
)

# 4. Validate and test templates
def test_template(template, test_data):
    """Test a template with sample data."""
    try:
        result = template.format(**test_data)
        print("✓ Template test passed")
        print(f"Result preview: {result[:100]}...")
    except Exception as e:
        print(f"✗ Template test failed: {e}")

test_template(
    flexible_template,
    {"content_type": "article", "topic": "AI ethics"}
)

## Real-World Example: Customer Support System

Let's build a complete customer support prompt system:

In [ ]:
class CustomerSupportTemplates:
    """Prompt templates for customer support system."""
    
    # Initial response template
    initial_response = PromptTemplate(
        """You are a helpful customer support agent for {company_name}.
        
        Customer Profile:
        - Name: {customer_name}
        - Account Type: {account_type}
        - Previous Issues: {previous_issues}
        
        Current Issue: {issue_description}
        
        Provide a helpful, empathetic response that:
        1. Acknowledges their concern
        2. Provides clear next steps
        3. Sets expectations for resolution time
        
        Tone: {tone}""",
        tone="professional and friendly",
        previous_issues="None"
    )
    
    # Escalation template
    escalation = PromptTemplate(
        """Summarize this support ticket for escalation to {department}:
        
        Customer: {customer_name} ({account_type})
        Issue: {issue_description}
        Previous attempts: {previous_attempts}
        Customer sentiment: {sentiment}
        
        Create a brief summary including:
        - Key issue
        - What's been tried
        - Recommended action
        - Urgency level"""
    )
    
    # Resolution follow-up
    follow_up = PromptTemplate(
        """Draft a follow-up message for a resolved support ticket:
        
        Customer: {customer_name}
        Original issue: {issue_description}
        Resolution: {resolution}
        Time to resolve: {resolution_time}
        
        Include:
        - Confirmation that issue is resolved
        - Brief summary of what was done
        - Invitation for feedback
        - Preventive tips if applicable"""
    )

# Use the support system
support = CustomerSupportTemplates()

# Handle a customer issue
issue_data = {
    "company_name": "TechCorp",
    "customer_name": "Sarah Johnson",
    "account_type": "Premium",
    "issue_description": "Unable to access dashboard, getting error 500"
}

initial_prompt = support.initial_response.format(**issue_data)
response = client.complete(initial_prompt)

print("Customer Support Response:")
print(response)
print("\n" + "="*50 + "\n")

# Later, escalate if needed
escalation_data = {
    "department": "Technical Team",
    "customer_name": "Sarah Johnson",
    "account_type": "Premium",
    "issue_description": "Dashboard access error 500",
    "previous_attempts": "Cleared cache, tried different browser, reset password",
    "sentiment": "Frustrated but polite"
}

escalation_prompt = support.escalation.format(**escalation_data)
escalation_summary = client.complete(escalation_prompt)

print("Escalation Summary:")
print(escalation_summary)

## Summary

In this tutorial, you learned:

- ✅ How to create basic prompt templates with variables
- ✅ How to use default values for optional parameters
- ✅ How to validate templates and handle errors
- ✅ How to compose templates for complex scenarios
- ✅ How to create dynamic and conditional templates
- ✅ How to organize templates in libraries
- ✅ Best practices for prompt engineering

Prompt templates are essential for building maintainable AI applications. They help you:
- Separate prompt logic from application code
- Reuse proven prompt patterns
- Maintain consistency across your application
- Test and iterate on prompts easily

## Next Steps

Continue your AILib journey with:

- **Tutorial 4: Prompt Builder** - Build complex conversations programmatically
- **Tutorial 5: Session Management** - Maintain conversation state
- **Tutorial 6: Chains** - Connect multiple operations sequentially

Happy templating! 🎯